In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from googletrans import Translator
from pandas import Series
from pandas import DataFrame
%matplotlib inline


In [2]:
item_category_df = pd.read_csv("data/item_categories.csv")
items_df = pd.read_csv("data/items.csv")
shops_df = pd.read_csv("data/shops.csv")
sample_df = pd.read_csv("data/sample_submission.csv")
train_df = pd.read_csv("data/sales_train_v2.csv")
test_df = pd.read_csv("data/test.csv")

# date-> day, month, year

In [3]:
f = lambda x: x.split(".")
train_df['date'] =list(map(f, train_df['date']))

f = lambda x: x[0]
train_df['day'] =list(map(f, train_df['date']))

f = lambda x: x[1]
train_df['month'] =list(map(f, train_df['date']))

f = lambda x: x[2]
train_df['year'] =list(map(f, train_df['date']))

In [4]:
train_df = train_df.drop(['date'],axis=1)
train_df.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year
0,0,59,22154,999.00,1.0,02,01,2013
1,0,25,2552,899.00,1.0,03,01,2013
2,0,25,2552,899.00,-1.0,05,01,2013
3,0,25,2554,1709.05,1.0,06,01,2013
4,0,25,2555,1099.00,1.0,15,01,2013


In [5]:
train_df = train_df.drop(['day'],axis=1)

In [6]:
y_data = train_df.item_cnt_day.reshape(train_df.item_cnt_day.size,1)

/home/rosa/miniconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.


In [7]:
train_df = train_df.drop(['item_cnt_day'],axis=1)

# Promotion

### 같은 가게 같은 아이템의 가격의 어느 순간 하락하면 프로모션이 진행되었다고 판단
### 프로모션의 여부 피쳐를 추가해준다.
### 주의. 가격이 상승함은 조건에서 걸러준다

In [8]:
pro_df = train_df.groupby(["shop_id","item_id","month","year","date_block_num"]).agg({"item_price":"sum"}).unstack().fillna(0)
pro_df.head()

item_price                                        \
date_block_num                     0       1    2    3    4    5    6    7    
shop_id item_id month year                                                    
0       30      02    2013        0.0  2385.0  0.0  0.0  0.0  0.0  0.0  0.0   
        31      02    2013        0.0  3038.0  0.0  0.0  0.0  0.0  0.0  0.0   
        32      01    2013      884.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                02    2013        0.0  1547.0  0.0  0.0  0.0  0.0  0.0  0.0   
        33      01    2013     1041.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                                     ...                                      \
date_block_num               8    9  ...    24   25   26   27   28   29   30   
shop_id item_id month year           ...                                       
0       30      02    2013  0.0  0.0 ...   0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        31      02    2013  0.0  0.0 ...   0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        32      01    2013  0.0  0.0 ...   0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                02    2013  0.0  0.0 ...   0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        33      01    2013  0.0  0.0 ...   0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                                           
date_block_num               31   32   33  
shop_id item_id month year                 
0       30      02    2013  0.0  0.0  0.0  
        31      02    2013  0.0  0.0  0.0  
        32      01    2013  0.0  0.0  0.0  
                02    2013  0.0  0.0  0.0  
        33      01    2013  0.0  0.0  0.0  

[5 rows x 34 columns]

In [9]:
pro_min_df = train_df.groupby(["shop_id","item_id","month","year","date_block_num"]).agg({"item_price":"min"}).unstack().fillna(0)
pro_min_df.head()

item_price                                       \
date_block_num                     0      1    2    3    4    5    6    7    
shop_id item_id month year                                                   
0       30      02    2013        0.0  265.0  0.0  0.0  0.0  0.0  0.0  0.0   
        31      02    2013        0.0  434.0  0.0  0.0  0.0  0.0  0.0  0.0   
        32      01    2013      221.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                02    2013        0.0  221.0  0.0  0.0  0.0  0.0  0.0  0.0   
        33      01    2013      347.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                                     ...                                      \
date_block_num               8    9  ...    24   25   26   27   28   29   30   
shop_id item_id month year           ...                                       
0       30      02    2013  0.0  0.0 ...   0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        31      02    2013  0.0  0.0 ...   0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        32      01    2013  0.0  0.0 ...   0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                02    2013  0.0  0.0 ...   0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        33      01    2013  0.0  0.0 ...   0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                                           
date_block_num               31   32   33  
shop_id item_id month year                 
0       30      02    2013  0.0  0.0  0.0  
        31      02    2013  0.0  0.0  0.0  
        32      01    2013  0.0  0.0  0.0  
                02    2013  0.0  0.0  0.0  
        33      01    2013  0.0  0.0  0.0  

[5 rows x 34 columns]

In [10]:
pro_max_df = train_df.groupby(["shop_id","item_id","month","year","date_block_num"]).agg({"item_price":"max"}).unstack().fillna(0)
pro_max_df.head()

item_price                                       \
date_block_num                     0      1    2    3    4    5    6    7    
shop_id item_id month year                                                   
0       30      02    2013        0.0  265.0  0.0  0.0  0.0  0.0  0.0  0.0   
        31      02    2013        0.0  434.0  0.0  0.0  0.0  0.0  0.0  0.0   
        32      01    2013      221.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                02    2013        0.0  221.0  0.0  0.0  0.0  0.0  0.0  0.0   
        33      01    2013      347.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                                     ...                                      \
date_block_num               8    9  ...    24   25   26   27   28   29   30   
shop_id item_id month year           ...                                       
0       30      02    2013  0.0  0.0 ...   0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        31      02    2013  0.0  0.0 ...   0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        32      01    2013  0.0  0.0 ...   0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                02    2013  0.0  0.0 ...   0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        33      01    2013  0.0  0.0 ...   0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                                           
date_block_num               31   32   33  
shop_id item_id month year                 
0       30      02    2013  0.0  0.0  0.0  
        31      02    2013  0.0  0.0  0.0  
        32      01    2013  0.0  0.0  0.0  
                02    2013  0.0  0.0  0.0  
        33      01    2013  0.0  0.0  0.0  

[5 rows x 34 columns]

In [11]:
pro_min_df.equals(pro_max_df) # 가격의 변동이 있다

False

In [12]:
pro_df = pro_min_df.where((pro_min_df == pro_max_df), other = np.nan)  # 최대와 최소 값 비교, 다르면 nan

In [13]:
null_columns=pro_df.columns[pro_df.isnull().any()]
sum(pro_df[null_columns].isnull().sum()) #총 nan값

94586

In [14]:
diffenent_price = (pro_df[pro_df.isnull().any(axis=1)][null_columns]) 

In [15]:
diffenent_price["promotion"] =np.ones(len(diffenent_price))
diffenent_price.head()

item_price                                          \
date_block_num                      0    1    2    3    4    5    6    7    8   
shop_id item_id month year                                                      
0       51      01    2013        NaN  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        2678    02    2013        0.0  NaN  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        2748    01    2013        NaN  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        2761    01    2013        NaN  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        3320    01    2013        NaN  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                                   ...                                        \
date_block_num                9    ...      25   26   27   28   29   30   31   
shop_id item_id month year         ...                                         
0       51      01    2013  0.0    ...     0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        2678    02    2013  0.0    ...     0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        2748    01    2013  0.0    ...     0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        2761    01    2013  0.0    ...     0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        3320    01    2013  0.0    ...     0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                                     promotion  
date_block_num               32   33            
shop_id item_id month year                      
0       51      01    2013  0.0  0.0       1.0  
        2678    02    2013  0.0  0.0       1.0  
        2748    01    2013  0.0  0.0       1.0  
        2761    01    2013  0.0  0.0       1.0  
        3320    01    2013  0.0  0.0       1.0  

[5 rows x 35 columns]

In [16]:
diffenent_price = diffenent_price.drop("item_price",axis =1 )

In [17]:
diffenent_price_2 = diffenent_price.reset_index()
diffenent_price_2.head()

,shop_id,item_id,month,year,promotion
date_block_num,,,,,
0,0,51,01,2013,1.0
1,0,2678,02,2013,1.0
2,0,2748,01,2013,1.0
3,0,2761,01,2013,1.0
4,0,3320,01,2013,1.0


In [18]:
train_df.head()
len(train_df)

2935849

In [58]:
df =pd.merge(train_df, diffenent_price_2, on=['shop_id','item_id','month','year'], how='outer').fillna(0)
print(len(df))
df.head()

/home/rosa/miniconda3/envs/py36/lib/python3.6/site-packages/pandas/core/reshape/merge.py:558: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/home/rosa/miniconda3/envs/py36/lib/python3.6/site-packages/pandas/core/generic.py:2530: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


2935849


,date_block_num,shop_id,item_id,item_price,item_cnt_day,month,year,"(promotion, )"
0,0,59,22154,999.00,1.0,01,2013,0.0
1,0,25,2552,899.00,1.0,01,2013,0.0
2,0,25,2552,899.00,-1.0,01,2013,0.0
3,0,25,2554,1709.05,1.0,01,2013,0.0
4,0,25,2555,1099.00,1.0,01,2013,0.0


In [59]:
df.columns.values[-1] = 'promotion'

In [64]:
df.head()

,date_block_num,shop_id,item_id,item_price,month,year,promotion
0,0,59,22154,999.00,01,2013,NaN
1,0,25,2552,899.00,01,2013,NaN
2,0,25,2552,899.00,01,2013,NaN
3,0,25,2554,1709.05,01,2013,NaN
4,0,25,2555,1099.00,01,2013,NaN
5,0,25,2564,349.00,01,2013,NaN
6,0,25,2565,549.00,01,2013,NaN
7,0,25,2572,239.00,01,2013,NaN
8,0,25,2572,299.00,01,2013,NaN
9,0,25,2573,299.00,01,2013,NaN


In [37]:
df.isnull().values.any()

True

## ==============전처리 끝===================

In [38]:
y_data = df.item_cnt_day.reshape(df.item_cnt_day.size,1)

AttributeError: 'DataFrame' object has no attribute 'item_cnt_day'

In [62]:
df = df.drop(['item_cnt_day'],axis=1)

In [30]:
y_data.shape

(2935849, 1)

In [31]:
x_data = df.values

In [32]:
x_data.shape

(2935849, 7)

In [33]:
from sklearn import preprocessing

minmax_scale = preprocessing.MinMaxScaler(feature_range=(0,5)).fit(x_data)
x_scaled_data = minmax_scale.transform(x_data)

x_scaled_data[:3]

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x_scaled_data, y_data, test_size=0.33)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
from sklearn import  linear_model

regr = linear_model.LinearRegression(n_jobs=1)
regr.fit(X_train, y_train)
regr

In [ ]:
print('Coefficients: ', regr.coef_) # w1
print('intercept: ', regr.intercept_) #w0

# Metric 측정

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [ ]:
y_true = y_test
y_hat = regr.predict(X_test)

r2_score(y_true, y_hat), mean_absolute_error(y_true, y_hat), mean_squared_error(y_true, y_hat)
# r2_score : 1에 가까우면 좋음  

In [ ]:
y_true = y_train
y_hat = regr.predict(X_train)

r2_score(y_true, y_hat), mean_absolute_error(y_true, y_hat), mean_squared_error(y_true, y_hat)